In [1]:
# Run this in a virtual environment. 
# Currently, I'm running this on Columbia's Insomnia HPC Cluster for the GPUs

# To get a python venv to be a ipykernel, run:
# python -m ipykernel install --user --name=env --display-name "Virtual Env"

!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install vllm

In [2]:
# Run this in a terminal: 
# (Wait for it to finish running before continuing)

# vllm serve ai21labs/AI21-Jamba-Reasoning-3B   --trust-remote-code   --max-model-len 16384   --tensor-parallel-size 1

In [19]:
import requests

def check_vllm_status():
    try:
        response = requests.get("http://localhost:8000/health")
        if response.status_code == 200:
            print("vllm server is running")
            return True
    except requests.exceptions.ConnectionError:
        print("vllm server is not running")
        return False

try:
    # Monitor the process
    while True:
        if check_vllm_status() == True:
            print("The vllm server is ready to serve.")
            break
        else:
            print("The vllm server has stopped.")
            stdout, stderr = vllm_process.communicate(timeout=10)
            print(f"STDOUT: {stdout.decode('utf-8')}")
            print(f"STDERR: {stderr.decode('utf-8')}")
            break
        time.sleep(5)  # Check every second
except KeyboardInterrupt:
    print("Stopping the check of vllm...")

vllm server is running
The vllm server is ready to serve.


In [9]:
import requests
import json
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from fastapi.responses import StreamingResponse
import requests

def ask_model(question: str, model: str):
    """
    Sends a request to the model server and fetches a response.
    """
    url = "http://localhost:8000/v1/chat/completions"  # Adjust the URL if different
    headers = {"Content-Type": "application/json"}
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": question
            }
        ]
    }

    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()  # Raise exception for HTTP errors
    return response.json()

# Return answer when ready
def ask_model_conversation(messages: list, model: str):
    """
    Sends the entire message history to the model server.
    """
    url = "http://localhost:8000/v1/chat/completions"  # Adjust if different
    headers = {"Content-Type": "application/json"}
    
    data = {
        "model": model,
        "messages": messages  # Send the whole history
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise exception for HTTP errors
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API Error: {e}")
        return None

def get_clean_answer(content: str):
    """
    Parses the <think> tags out of the model's response.
    """
    try:
        final_answer = content.split("</think>")[-1].strip()
    except Exception as e:
        final_answer = content.strip() # Fallback
    return final_answer

# Stream answer token by token
def stream_llm_response(question:str, model:str):
    url = "http://localhost:8000/v1/chat/completions"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": model,
        "messages": [{"role": "user", "content": question}],
        "stream": True  # 🔥 Enable streaming
    }

    with requests.post(url, headers=headers, json=data, stream=True) as response:
        for line in response.iter_lines():
            if line:
                # OpenAI-style streaming responses are prefixed with "data: "
                decoded_line = line.decode("utf-8").replace("data: ", "")
                yield decoded_line + "\n"

In [5]:
# Test Usage:
result = ask_model("What is the best type of architecture for Small LMs and Large LMs? \
Should I use a transformer based model or linear attention or state space model or other architecture?",\
                   "ai21labs/AI21-Jamba-Reasoning-3B")

# If you want the whole JSON object:
# print(json.dumps(result, indent=2))

In [10]:
# Get a clean answer
import json

# Assuming 'result' is the dictionary you got from the model
content = result["choices"][0]["message"]["content"]

# Split the content at the </think> tag and take the last part
# .strip() cleans up any leading/trailing newlines
try:
    final_answer = content.split("</think>")[-1].strip()
except Exception as e:
    # Fallback in case </think> isn't present
    final_answer = content.strip()

# print(final_answer)

# Make it pretty with markdown
from IPython.display import display, Markdown

# Use the 'final_answer' from Solution 1
display(Markdown(final_answer))

The choice of architecture for small vs. large language models (LMs) depends on the trade‑off between computational efficiency, contextual depth, and practicality.  

| Size | Typical Architecture | Why It Fits |
|------|----------------------|-------------|
| **Small LMs** (e.g., BERT, RoBERTa) | **Transformer‑based** (self‑attention) | • Fast parallelisation with GPUs/TPUs.<br>• Captures local syntax/semantics effectively.<br>• Widely used in pre‑trained models. |
| **Large LMs** (e.g., LLaMA, GPT‑3, GPT‑4) | **Transformer‑based** (self‑attention + optimisations) | • Still uses self‑attention for long‑range dependencies.<br>• Incorporates memory‑efficient variants (e.g., Linformer, Longformer) for longer contexts. |

**Key considerations:**

1. **Small LMs**  
   - *Transformer*: The gold standard for short to medium‑length tasks.  
   - *Efficiency hacks*: Use a smaller model size (e.g., 12‑16 layers) or quantisation to reduce inference latency.

2. **Large LMs**  
   - *Transformer*: The base architecture is still dominant, but many systems add:
     - *Linear attention* (reduces quadratic self‑attention complexity).
     - *Sparse attention* (keeps attention within a small window).
     - *State‑space variants* (RNNs or transformers with memory‑aware layers) for very long sequences.
   - *Efficiency*: Linear‑attention‑based models (Linformer, Longformer) are now competitive for long‑context generation.

**Bottom line:**  
For **small LMs**, a standard transformer is the best choice. For **large LMs**, stay with the transformer family but leverage optimisations (memory‑efficient attention, sparse layers) to keep inference fast and scalable. If you need even longer contexts, consider a hybrid model (e.g., a transformer with a state‑space core for the early part).

In [18]:
# --- MAIN CONVERSATIONAL LOOP ---

def start_chat(model_name: str):
    # This is the "memory" of the conversation
    chat_history = [] 
    
    print(f"Starting chat with {model_name}. Type 'quit' to exit.")
    
    while True:
        # 1. Get user input
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Chat ended.")
            break
            
        # 2. Append user message to history
        chat_history.append({
            "role": "user",
            "content": user_input
        })
        
        # 3. Call the API with the full history
        result = ask_model_conversation(chat_history, model_name)
        
        if result:
            # 4. Get the full assistant response
            full_assistant_message = result["choices"][0]["message"]["content"]
            
            # 5. Append the *full* response to history for context
            # This is crucial so the model "remembers" its own thoughts
            chat_history.append({
                "role": "assistant",
                "content": full_assistant_message
            })
            
            # 6. Get and display the clean answer
            clean_answer = get_clean_answer(full_assistant_message)
            
            # Using print() for a terminal. 
            # Use display(Markdown(clean_answer)) if in a notebook.
            # print(f"\nAssistant:\n{clean_answer}\n")
            display(Markdown(clean_answer))
        else:
            # If the API call failed, remove the last user message
            chat_history.pop()

# --- To run it ---
if __name__ == "__main__":
    MODEL_NAME = "ai21labs/AI21-Jamba-Reasoning-3B"
    start_chat(MODEL_NAME)

Starting chat with ai21labs/AI21-Jamba-Reasoning-3B. Type 'quit' to exit.


You:  What is AI21 Labs?


**AI21 Labs** is a technology company founded by Andrew Ng and a group of researchers from OpenAI.  It focuses on building state‑of‑the‑art large language models, advancing AI research, and providing robust AI‑powered tools for developers, researchers, and businesses.  

The most well‑known product of AI21 Labs is **AI21**, a large‑scale language model that powers a wide range of applications, from chatbots and code generation to creative writing and data analysis.  The company also develops open‑source frameworks and cloud platforms that enable developers to deploy and scale AI solutions efficiently.  

In essence, AI21 Labs bridges cutting‑edge AI research with practical, accessible tools to help users harness the power of large language models in a variety of domains.

You:  quit


Chat ended.
